# 使用HMM模型对NLTK自带的Brown词库进行词性标注

> 任务简述

假设我们的单词集： words = w1 ... wN

Tag集： tags = t1 ... tN

P(tags | words) 正比于 $P(t_i | t_{i-1}) * P(w_i | t_i)$

为了找一个句子的tag,其实就是找的最好的一套tags，让其最能够符合给定的单词(words)。

In [1]:
import nltk
import sys
nltk.download('brown')
from nltk.corpus import brown

[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\Gary\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\brown.zip.


## 词库预处理
给词加上开始和结束符号。

Brown里面的句子都是自己标注好了的，比如：(I , NOUN), (LOVE, VERB), (YOU, NOUN)

因此我们的开始符号也得跟他的格式符合：(START, START) (END, END)

In [3]:
brown_tags_words = []
for sent in brown.tagged_sents():
    # 先加开头
    brown_tags_words.append(("START", "START"))
    # 为了简单起见，我们把tag都省略成前两个字母
    brown_tags_words.extend([(tag[:2], word) for (word, tag) in sent])
    # 加个结尾
    brown_tags_words.append(("END", "END"))

brown_tags_words

[('START', 'START'),
 ('AT', 'The'),
 ('NP', 'Fulton'),
 ('NN', 'County'),
 ('JJ', 'Grand'),
 ('NN', 'Jury'),
 ('VB', 'said'),
 ('NR', 'Friday'),
 ('AT', 'an'),
 ('NN', 'investigation'),
 ('IN', 'of'),
 ('NP', "Atlanta's"),
 ('JJ', 'recent'),
 ('NN', 'primary'),
 ('NN', 'election'),
 ('VB', 'produced'),
 ('``', '``'),
 ('AT', 'no'),
 ('NN', 'evidence'),
 ("''", "''"),
 ('CS', 'that'),
 ('DT', 'any'),
 ('NN', 'irregularities'),
 ('VB', 'took'),
 ('NN', 'place'),
 ('.', '.'),
 ('END', 'END'),
 ('START', 'START'),
 ('AT', 'The'),
 ('NN', 'jury'),
 ('RB', 'further'),
 ('VB', 'said'),
 ('IN', 'in'),
 ('NN', 'term-end'),
 ('NN', 'presentments'),
 ('CS', 'that'),
 ('AT', 'the'),
 ('NN', 'City'),
 ('JJ', 'Executive'),
 ('NN', 'Committee'),
 (',', ','),
 ('WD', 'which'),
 ('HV', 'had'),
 ('JJ', 'over-all'),
 ('NN', 'charge'),
 ('IN', 'of'),
 ('AT', 'the'),
 ('NN', 'election'),
 (',', ','),
 ('``', '``'),
 ('VB', 'deserves'),
 ('AT', 'the'),
 ('NN', 'praise'),
 ('CC', 'and'),
 ('NN', 'thanks'),


## 统计

把我们所有的词库中拥有的单词与tag之间的关系，做个简单粗暴的统计。

$P(w_i | t_i) = count(w_i, t_i) / count(t_i)$

这里NLTK库给了我们做统计的工具。

In [4]:
# 状态频率分布
cfd_tagwords = nltk.ConditionalFreqDist(brown_tags_words)
# 状态概率分布
cpd_tagwords = nltk.ConditionalProbDist(cfd_tagwords, nltk.MLEProbDist)

print("The probability of an adjective (JJ) being 'new' is", cpd_tagwords["JJ"].prob("new"))
print("The probability of a verb (VB) being 'duck' is", cpd_tagwords["VB"].prob("duck"))

The probability of an adjective (JJ) being 'new' is 0.01472344917632025
The probability of a verb (VB) being 'duck' is 6.042713350943527e-05


接下来计算马尔科夫链:
$P(t_i | t_{i-1}) = count(t_{i-1}, t_i) / count(t_{i-1})$

In [5]:
# 去除所有tag
brown_tags = [tag for (tag, word) in brown_tags_words ]

# count(t{i-1} ti), bigram:前后两个一组，联在一起
cfd_tags= nltk.ConditionalFreqDist(nltk.bigrams(brown_tags))
# P(ti | t{i-1})
cpd_tags = nltk.ConditionalProbDist(cfd_tags, nltk.MLEProbDist)

print("If we have just seen 'DT', the probability of 'NN' is", cpd_tags["DT"].prob("NN"))
print( "If we have just seen 'VB', the probability of 'JJ' is", cpd_tags["VB"].prob("DT"))
print( "If we have just seen 'VB', the probability of 'NN' is", cpd_tags["VB"].prob("NN"))

If we have just seen 'DT', the probability of 'NN' is 0.5057722522030194
If we have just seen 'VB', the probability of 'JJ' is 0.016885067592065053
If we have just seen 'VB', the probability of 'NN' is 0.10970977711020183


## 结果探究

比如， 一句话，"I want to race"， 一套tag，"PP VB TO VB"
他们之间的匹配度有多高呢？

In [7]:
prob_tagsequence = cpd_tags["START"].prob("PP") * cpd_tagwords["PP"].prob("I") * cpd_tags["PP"].prob("VB") * cpd_tagwords["VB"].prob("want") * cpd_tags["VB"].prob("TO") * cpd_tagwords["TO"].prob("to") * cpd_tags["TO"].prob("VB") * cpd_tagwords["VB"].prob("race") * cpd_tags["VB"].prob("END")

print( "The probability of the tag sequence 'START PP VB TO VB END' for 'I want to race' is:", prob_tagsequence)

The probability of the tag sequence 'START PP VB TO VB END' for 'I want to race' is: 1.0817766461150474e-14


## 预测（解码）问题

如果我们手上有一句话，怎么知道最符合的tag是哪组呢？

首先，我们拿出所有tags（tags的全集）

In [8]:
distinct_tags = set(brown_tags)

随便写上一句话

In [9]:
sentence = ["I", "want", "to", "race" ]
sentlen = len(sentence)

接下来，开始维特比：

从1循环到句子的总长N，记为i

每次都找出以tag X为最终节点，长度为i的tag链

In [10]:
viterbi = [ ]

同时，还需要一个回溯器：

从1循环到句子的总长N，记为i

把所有tag X 前一个Tag记下来。

In [11]:
backpointer = [ ]

first_viterbi = { }
first_backpointer = { }
for tag in distinct_tags:
    # don't record anything for the START tag
    if tag == "START": continue
    first_viterbi[ tag ] = cpd_tags["START"].prob(tag) * cpd_tagwords[tag].prob( sentence[0] )
    first_backpointer[ tag ] = "START"

print(first_viterbi)
print(first_backpointer)

{',': 0.0, 'IN': 0.0, '(': 0.0, 'RN': 0.0, "''": 0.0, '.': 0.0, 'PN': 0.0, 'NI': 3.3324520848931064e-07, 'DT': 0.0, ')-': 0.0, 'AT': 0.0, 'MD': 0.0, 'NP': 1.7319067623793952e-06, 'CS': 0.0, 'OD': 0.0, '*': 0.0, 'CC': 0.0, ',-': 0.0, 'UH': 0.0, '*-': 0.0, 'CD': 0.0, 'AP': 0.0, 'DO': 0.0, 'WQ': 0.0, 'QL': 0.0, 'BE': 0.0, ')': 0.0, 'HV': 0.0, 'AB': 0.0, '--': 0.0, 'RP': 0.0, 'WR': 0.0, 'WD': 0.0, 'RB': 0.0, 'TO': 0.0, ':': 0.0, ':-': 0.0, 'WP': 0.0, 'FW': 0.0, 'NN': 1.0580313619573935e-06, '``': 0.0, '.-': 0.0, 'VB': 0.0, "'": 0.0, 'PP': 0.014930900689060006, 'NR': 0.0, 'EX': 0.0, 'END': 0.0, '(-': 0.0, 'JJ': 0.0}
{',': 'START', 'IN': 'START', '(': 'START', 'RN': 'START', "''": 'START', '.': 'START', 'PN': 'START', 'NI': 'START', 'DT': 'START', ')-': 'START', 'AT': 'START', 'MD': 'START', 'NP': 'START', 'CS': 'START', 'OD': 'START', '*': 'START', 'CC': 'START', ',-': 'START', 'UH': 'START', '*-': 'START', 'CD': 'START', 'AP': 'START', 'DO': 'START', 'WQ': 'START', 'QL': 'START', 'BE': 'ST

以上，是所有的第一个viterbi 和第一个回溯点。

接下来，把楼上这些，存到Vitterbi和Backpointer两个变量里去

In [12]:
viterbi.append(first_viterbi)
backpointer.append(first_backpointer)

In [13]:
# 观察目前最好的tag
currbest = max(first_viterbi.keys(), key = lambda tag: first_viterbi[ tag ])
print( "Word", "'" + sentence[0] + "'", "current best two-tag sequence:", first_backpointer[ currbest], currbest)

Word 'I' current best two-tag sequence: START PP


In [15]:
for wordindex in range(1, len(sentence)):
    this_viterbi = { }
    this_backpointer = { }
    prev_viterbi = viterbi[-1]

    for tag in distinct_tags:
        # START没有卵用的，我们要忽略
        if tag == "START": continue

        # 如果现在这个tag是X，现在的单词是w，
        # 我们想找前一个tag Y，并且让最好的tag sequence以Y X结尾。
        # 也就是说
        # Y要能最大化：
        # prev_viterbi[ Y ] * P(X | Y) * P( w | X)

        best_previous = max(prev_viterbi.keys(),
                            key = lambda prevtag:
                                prev_viterbi[ prevtag ] * cpd_tags[prevtag].prob(tag) * cpd_tagwords[tag].prob(sentence[wordindex]))

        this_viterbi[ tag ] = prev_viterbi[ best_previous] * cpd_tags[ best_previous ].prob(tag) * cpd_tagwords[ tag].prob(sentence[wordindex])
        this_backpointer[ tag ] = best_previous

    # 每次找完Y 我们把目前最好的 存一下
    currbest = max(this_viterbi.keys(), key = lambda tag: this_viterbi[ tag ])
    print( "Word", "'" + sentence[ wordindex] + "'", "current best two-tag sequence:", this_backpointer[ currbest], currbest)


    # 完结
    # 全部存下来
    viterbi.append(this_viterbi)
    backpointer.append(this_backpointer)

Word 'want' current best two-tag sequence: PP VB
Word 'to' current best two-tag sequence: VB TO
Word 'race' current best two-tag sequence: IN NN


找END，结束：

In [16]:
# 找所有以END结尾的tag sequence
prev_viterbi = viterbi[-1]
best_previous = max(prev_viterbi.keys(),
                    key = lambda prevtag: prev_viterbi[ prevtag ] * cpd_tags[prevtag].prob("END"))

prob_tagsequence = prev_viterbi[ best_previous ] * cpd_tags[ best_previous].prob("END")

# 我们这会儿是倒着存的。。。。因为。。好的在后面
best_tagsequence = [ "END", best_previous ]
# 同理 这里也有倒过来
backpointer.reverse()

最后，回溯所有的回溯点

此时，最好的tag就是backpointer里面的current best

In [17]:
current_best_tag = best_previous
for bp in backpointer:
    best_tagsequence.append(bp[current_best_tag])
    current_best_tag = bp[current_best_tag]

显示结果：

In [21]:
best_tagsequence.reverse()
print( "The sentence was:", end = " ")
for w in sentence: print( w, end = " ")
print("\n")
print( "The best tag sequence is:", end = " ")
for t in best_tagsequence: print (t, end = " ")
print("\n")
print( "The probability of the best tag sequence is:", prob_tagsequence)

The sentence was: I want to race 

The best tag sequence is: END NN IN VB PP START 

The probability of the best tag sequence is: 5.71772824864617e-14


可以分析得出，结果并不理想，应当需要更多的语料信息。